In [27]:
import json
import os
from pathlib import Path

import yaml
from dotenv import find_dotenv, load_dotenv
from openai import OpenAI
from qdrant_client import QdrantClient

from database.utils import embed_text, get_context, search

load_dotenv(find_dotenv())

True

In [2]:
config_path = Path("./config.yaml")
with config_path.open("r") as file:
    config = yaml.safe_load(file)

Create a OpenAI client

In [3]:
qdrant_client = QdrantClient(
    url=os.environ["QDRANT_CLUSTER_URL"],
    api_key=os.environ["QDRANT_API_KEY"],
)

openai_client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

Prompts

In [17]:
system_prompt = """
Vi ste koristan asistent koji može da odgovori isključivo na pitanja vezana za pravne teme. 
Molimo vas da prilikom razgovora sa klijentom koristite jasan i direktan jezik kako bi informacije bile lako razumljive. 
Vaš zadatak je da identifikujete potrebe klijenta i na osnovu toga pružite najrelevantnije informacije. 
Kada pružate odgovore ili savete, naglasite iz kojeg tačno pravnog člana dolazi informacija i obezbedite link ka tom članu kako bi klijent mogao dodatno da se informiše. 
Cilj je da komunikacija bude efikasna i da klijent oseti da je u dobrim rukama.

- Razgovarajte jasno i poentirano.
- Identifikujte ključne informacije koje klijent traži.
- Koristite informacije samo iz pravnih članova datih u kontekstu.
- Primarni izvor odgovora treba da budu odredbe članova 1 do 287, jer su oni važeći u trenutku kada Vi dajete odgovor. Ako se pitanje korisnika odnosi na samostalne članove Zakona o radu koji se nalaze u zakonu posle člana 287, potrebno je da odgovorite da možete da pružate informacije samo o trenutno važećim verzijama propisa i da niste u mogućnosti da pružite pouzdan odgovor.
- Uvek navedite izvor informacija i pružite link ka članu ili članovima.
- Odgovorite na pitanje klijenta samo ukoliko imate tačnu informaciju o odgovoru, u suprotnom ljubazno se izvinite i uputite klijenta da se obrati advokatu Anji Berić (https://www.linkedin.com/in/anja-beric-150285vb/).
- Zapamtite da je vaša uloga da olakšate klijentu razumevanje pravnih procedura i da mu pružite korisne i tačne informacije.
"""

context_prompt = """
KONTEKST:

{context}

"""

query_prompt = """
Pitanje klijenta: {query}
"""

Retrive context

In [5]:
tests_path = "./database/test_queries.json"
with open(tests_path, "r", encoding="utf-8") as file:
    test_samples = json.loads(file.read())

In [28]:
# query = test_samples["hard"][3]["query"]
# embedding = test_samples["hard"][3]["embedding"]
query = "Kako se odredbe Zakona o radu u vezi porodiljskog odsustva primenjuju na žene koje su započele korišćenje porodiljskog odsustva 2005. godine?"
embedding_response = embed_text(
    client=openai_client, text=query, model=config["openai"]["embedding_model"]["name"]
)
embedding = embedding_response.data[0].embedding

search_results = search(
    client=qdrant_client,
    collection=config["collection"]["name"],
    query_vector=embedding,
    limit=5,
    with_vectors=True,
)
context = get_context(search_results=search_results)

In [29]:
print(f"User Query: {query}\n\n")
print(context)

User Query: Kako se odredbe Zakona o radu u vezi porodiljskog odsustva primenjuju na žene koje su započele korišćenje porodiljskog odsustva 2005. godine?


Naslov: Član 11[s1]
Link do člana: https://www.paragraf.rs/propisi/zakon_o_radu.html#clan_11%5Bs1%5D
Član 11[s1]. Zaposlena žena koja je započela korišćenje porodiljskog odsustva u skladu sa članom 94. Zakona o radu ("Službeni glasnik RS", broj 24/05) do dana stupanja na snagu ovog zakona - nastavlja da koristi pravo na porodiljsko odsustvo i odsustvo sa rada radi nege deteta u skladu sa odredbama tog člana. Pravo iz stava 1. ovog člana pripada i ocu deteta.


Naslov: Član 94
Link do člana: https://www.paragraf.rs/propisi/zakon_o_radu.html#clan_94
Član 94. Zaposlena žena ima pravo na odsustvo sa rada zbog trudnoće i porođaja (u daljem tekstu: porodiljsko odsustvo), kao i odsustvo sa rada radi nege deteta, u ukupnom trajanju od 365 dana. Zaposlena žena ima pravo da otpočne porodiljsko odsustvo na osnovu nalaza nadležnog zdravstvenog 

Format messagges

In [30]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": context_prompt.format(context=context)},
    {"role": "user", "content": query_prompt.format(query=query)},
]

Get response

In [31]:
response = openai_client.chat.completions.create(
    model=config["openai"]["gpt_model"]["name_light"],
    temperature=config["openai"]["gpt_model"]["temperature"],
    messages=messages,
)

In [32]:
print(response.choices[0].message.content)

Osobe koje su započele korišćenje porodiljskog odsustva pre stupanja na snagu određenih odredbi Zakona o radu zadržavaju pravo na korišćenje tog odsustva u skladu sa odredbama koje su bile na snazi u trenutku kada su ga započele. Dakle, žene koje su započele korišćenje porodiljskog odsustva 2005. godine nastavljaju da koriste to pravo u skladu sa odredbama Zakona o radu koje su važile u tom periodu. 

Konkretno, prema članu 11[s1] Zakona o radu, zaposlena žena koja je započela korišćenje porodiljskog odsustva u skladu sa članom 94 Zakona o radu ("Službeni glasnik RS", broj 24/05) do dana stupanja na snagu određenih odredbi zakona, nastavlja da koristi pravo na porodiljsko odsustvo i odsustvo sa rada radi nege deteta u skladu sa odredbama tog člana. Pravo iz ovog člana pripada i ocu deteta.

Dakle, žene koje su započele korišćenje porodiljskog odsustva 2005. godine i dalje imaju pravo na korišćenje tog odsustva u skladu sa odredbama Zakona o radu koje su bile na snazi u tom periodu. Za 